In [1]:
import os
import pandas as pd

# Define the directory containing the CSV files
overall_dir = "../final-results" 
model_dirs = [ f.path for f in os.scandir(overall_dir) if f.is_dir() ]
dirs = []
for folder in model_dirs:
    dirs.extend([ f.path for f in os.scandir(folder) if f.is_dir() ])



In [2]:
for filename in os.listdir(dirs[0]):
    if filename.endswith(".csv") and 'n_shots' in filename and "samples" not in filename:
        file_path = os.path.join(dirs[0], filename)
        break
    
df = pd.read_csv(file_path)
df

,Unnamed: 0,outputs,label_tokens,true_numeric_labels,true_label_present_in_prompt,correct,predicted_label_present_in_prompt,prompt_labels,prompt_example_indices,id,n_shots,run_number,unlimiformer,nspw
0,4847,compromised card,comprom,22,True,True,False,"{22: 3, 9: 1, 27: 1, 20: 1, 26: 1, 4: 1, 51: 1...","[4847, 2350, 1419, 9477, 5489, 2179, 5814, 835...",4847,25,5,False,25
1,2350,compromised card,comprom,22,True,True,False,"{22: 3, 9: 1, 27: 1, 20: 1, 26: 1, 4: 1, 51: 1...","[4847, 2350, 1419, 9477, 5489, 2179, 5814, 835...",2350,25,5,False,25
2,1419,card about to expire,card about,9,True,True,False,"{22: 3, 9: 1, 27: 1, 20: 1, 26: 1, 4: 1, 51: 1...","[4847, 2350, 1419, 9477, 5489, 2179, 5814, 835...",1419,25,5,False,25
3,9477,declined transfer,declined transfer,27,True,True,True,"{22: 3, 9: 1, 27: 1, 20: 1, 26: 1, 4: 1, 51: 1...","[4847, 2350, 1419, 9477, 5489, 2179, 5814, 835...",9477,25,5,False,25
4,5489,cash withdrawal not recognised,cash withdrawal not,20,True,True,False,"{22: 3, 9: 1, 27: 1, 20: 1, 26: 1, 4: 1, 51: 1...","[4847, 2350, 1419, 9477, 5489, 2179, 5814, 835...",5489,25,5,False,25
5,2179,declined cash withdrawal,declined c,26,True,True,False,"{22: 3, 9: 1, 27: 1, 20: 1, 26: 1, 4: 1, 51: 1...","[4847, 2350, 1419, 9477, 5489, 2179, 5814, 835...",2179,25,5,False,25
6,5814,automatic top up,automatic,4,True,True,False,"{22: 3, 9: 1, 27: 1, 20: 1, 26: 1, 4: 1, 51: 1...","[4847, 2350, 1419, 9477, 5489, 2179, 5814, 835...",5814,25,5,False,25
7,8355,Refund not showing up,Ref,51,True,True,False,"{22: 3, 9: 1, 27: 1, 20: 1, 26: 1, 4: 1, 51: 1...","[4847, 2350, 1419, 9477, 5489, 2179, 5814, 835...",8355,25,5,False,25
8,472,getting spare card,getting spare,39,True,True,False,"{22: 3, 9: 1, 27: 1, 20: 1, 26: 1, 4: 1, 51: 1...","[4847, 2350, 1419, 9477, 5489, 2179, 5814, 835...",472,25,5,False,25
9,4812,pending transfer,pending transfer,48,True,True,True,"{22: 3, 9: 1, 27: 1, 20: 1, 26: 1, 4: 1, 51: 1...","[4847, 2350, 1419, 9477, 5489, 2179, 5814, 835...",4812,25,5,False,25


In [3]:
def get_labelset_info(df):
    all_output_labels = list(set(df['label_tokens']))
    all_labels = {k: {'TP': 0, 'FP': 0, 'FN': 0} for k in list(set(df['outputs']))}
    # need to track precision and recall for each class 
    def starts_any(prefix, longlist):
        for i in longlist:
            if i.startswith(prefix):
                return i
        return False

    for index, example in df.iterrows():
        if example['correct']:
            # this is a TP for one class 
            all_labels[example['outputs']]['TP'] = all_labels[example['outputs']]['TP'] + 1
        else:
            # this is a FP for the class we predicted...  
            all_labels[example['outputs']]['FP'] = all_labels[example['outputs']]['FP'] + 1       
            
            # ... and a FN for the true class
            if key := starts_any(example['label_tokens'], all_labels.keys()):
                all_labels[key]['FN'] = all_labels[key]['FN'] + 1
            else:
                #print(f"WARNING: creating new key for label {example['label_tokens']}")
                all_labels[example['label_tokens']] = {'TP': 0, 'FP': 0, 'FN': 1} 
    return all_labels

In [4]:
def calc_f1(df, labelsize):
    labelset = get_labelset_info(df)
    sum_f1s = 0
    for label in labelset:
        count = labelset[label]
        if count['TP'] == 0:
            p = r = f1 = 0
        else:
            p = count['TP']/(count['TP'] + count['FP'])
            r = count['TP']/(count['TP'] + count['FN'])
            f1 = 2* ((p*r)/(p+r))
        sum_f1s += f1
    #if len(labelset) not in [6, 50, 68, 77, 151]:
    #    print(f"warning: {len(labelset)} is unexpected value!")
    return sum_f1s/labelsize 
    
calc_f1(df, 77)

0.23376623376623376

In [10]:
print(os.listdir(dirs[0]))

['nspw=100_n_shots_results_100-outputs.pkl', 'nspw=32_+n_shots=32+run=5.csv', 'nspw=32_+n_shots=32+run=2.csv', 'nspw=10_+n_shots=10+run=0.csv', 'nspw=25_+n_shots=25+run=7.csv', 'nspw=10_+n_shots=10+run=7.csv', 'nspw=25_+n_shots=25+run=0.csv', 'nspw=10_+n_shots=10+run=9.csv', 'nspw=25_+n_shots=25+run=9.csv', 'nspw=64_+n_shots=64+run=0.csv', 'nspw=5_+n_shots=5+run=9.csv', 'nspw=64_+n_shots=64+run=7.csv', 'nspw=64_+n_shots=64+run=9.csv', 'nspw=5_+n_shots=5+run=0.csv', 'nspw=5_+n_shots=5+run=7.csv', 'nspw=100_+n_shots=100+run=4.csv', 'nspw=50_+n_shots=50+run=6.csv', 'nspw=64_n_shots_results_64.npy', '+n_shots=1+run=7.csv', 'nspw=32_n_shots_results_32.npy', '+n_shots=1+run=0.csv', 'nspw=100_+n_shots=100+run=3.csv', 'nspw=50_+n_shots=50+run=1.csv', 'nspw=5-n_shots.txt', 'nspw=50_+n_shots=50+run=8.csv', '+n_shots=1+run=9.csv', 'nspw=25_+n_shots=25+run=8.csv', 'n_shots_results_1.npy', 'nspw=5_n_shots_results_5.npy', 'nspw=10_+n_shots=10+run=8.csv', 'nspw=10_+n_shots=10+run=6.csv', 'nspw=25_+n_

In [5]:
from tqdm import tqdm

for directory in dirs:
    results = [] # Create an empty list to store results

    # Function to calculate the percentage of correct values
    def calculate_percentage(file_path):
        # Load the CSV file into a pandas DataFrame
        df = pd.read_csv(file_path)

        # Count the number of rows where 'correct' column is True
        correct_count = df['correct'].sum()
        
        # Calculate the percentage of correct values
        total_rows = len(df.index)
        #assert total_rows == 250, f"Caution: not the expected test set size, test set size is {total_rows} for file {file_path}"
        percentage_correct = (correct_count / total_rows) * 100
        if percentage_correct > 100:
            print(correct_count)
            print(total_rows)

        def get_labelsize(filename):
            half = 1
            if "half" in filename:
                half=2
                
            size = 0
            if "trecfine" in filename:
                size = 50 // half
            elif "trec" in filename:
                size = 6 // half
            elif "nlu" in filename:
                size = 68 // half
            elif "banking" in filename:
                size = 77 // half
            elif "clinic" in filename:
                size = 151 // half
            elif "tacred" in filename:
                size = 42
            else:
                assert False, filename
            return size

        return percentage_correct, calc_f1(df, labelsize=get_labelsize(file_path))

    # Iterate through each file in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".csv") and 'n_shots' in filename and "samples" not in filename:
            file_path = os.path.join(directory, filename)
            
            # Extracting information from filename
            parts = filename.split("_")
            if "nspw=" in parts[0]:
                nspw = parts[0][len("nspw="):]
                n_shots = parts[2].split("shots=")[1].split("+")[0]
                if nspw != n_shots:
                    #print(f"mismatch between {nspw} and {n_shots}, skipping")
                    continue
            else:
                nspw = parts[1].split('+')[0][6:]
            parts_remaining = parts[-1].split("+run=")
            n_shots = nspw
            if n_shots == '':
                print(parts)
                assert False
            if len(parts_remaining) < 2:
                assert False; continue
            run_num = parts_remaining[1].split(".csv")[0]
            
            
            # Calculate the percentage of correct values
            percentage_correct, f1 = calculate_percentage(file_path)
            
            # Append results to the list
            results.append([n_shots, percentage_correct, run_num, nspw, f1])

    # Create a DataFrame from the results list
    result_df = pd.DataFrame(results, columns=['n_shots', 'accuracy', 'run_num', 'nspw', 'f1'])
    output_filename = 'combined_results.csv'
    output_file_path = os.path.join(directory, output_filename)
    if "retrieval" in output_file_path:
        print(output_file_path)
    result_df.to_csv(output_file_path, index=False)


In [9]:
!cat /data/tir/projects/tir4/users/abertsch/allshot-prompting/outputs-llama/meta-llama+Llama-2-7b-hf/banking77/combined_results.csv


cat: /data/tir/projects/tir4/users/abertsch/allshot-prompting/outputs-llama/meta-llama+Llama-2-7b-chat-hf/banking77/combined_results.csv: No such file or directory
